In [117]:
import cv2
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, ConvLSTM2D, BatchNormalization
from keras.preprocessing.image import img_to_array

In [118]:
# Load pre-trained models
age_net = cv2.dnn.readNetFromCaffe('age_deploy.prototxt', 'age_net.caffemodel')
gender_net = cv2.dnn.readNetFromCaffe('gender_deploy.prototxt', 'gender_net.caffemodel')
face_net = cv2.dnn.readNetFromTensorflow('opencv_face_detector_uint8.pb', 'opencv_face_detector.pbtxt')

In [119]:
# Define model labels
age_labels = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
gender_labels = ['Male', 'Female']

In [120]:
# Preprocessing function
def preprocess_image(image):
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
    return blob

In [121]:
# Detect face
def detect_face(image):
    face_net.setInput(preprocess_image(image))
    detections = face_net.forward()
    h, w = image.shape[:2]
    faces = []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype('int')
            face = image[y:y1, x:x1]
            faces.append((x, y, x1, y1, face))

    return faces

In [122]:
# Predict age and gender
def predict_age_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746))

    gender_net.setInput(blob)
    gender = gender_labels[np.argmax(gender_net.forward())]

    age_net.setInput(blob)
    age = age_labels[np.argmax(age_net.forward())]

    return gender, age

In [123]:
# Emotion detection model
emotion_model = load_model('emotion_model.hdf5')

In [124]:
# Predict emotion
def predict_emotion(face):
    gray_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    resized_face = cv2.resize(gray_face, (64, 64)) / 255.0  
    reshaped_face = resized_face.reshape(1, 64, 64, 1)  
    emotion_prediction = emotion_model.predict(reshaped_face)
    emotion = np.argmax(emotion_prediction)
    return emotion

In [125]:
# Load image and run detection
image = cv2.imread('images.jpg')
faces = detect_face(image)

for (x, y, x1, y1, face) in faces:
    gender, age = predict_age_gender(face)
    #emotion = predict_emotion(face)

    label = f"{gender}, {age}"
    cv2.rectangle(image, (x, y), (x1, y1), (0, 255, 0), 2)
    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

In [126]:
# Display image
cv2.imshow('Gender, Age', image)
cv2.waitKey(0)
cv2.destroyAllWindows()